In [ ]:
%%capture
!pip install librosa==0.8.1
!pip install gdown

In [3]:
# json response
!gdown --id 1OsWBkQp610873Eb_d8QbYG6f7DGXbvXU
# json um_uh.mp3
!gdown --id 1vA66CXTxx2Kugyv-O2P_BpG8H1USZAhp

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OsWBkQp610873Eb_d8QbYG6f7DGXbvXU
To: /kaggle/working/response_1680775446119.json
100%|██████████████████████████████████████| 4.44k/4.44k [00:00<00:00, 4.58MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1vA66CXTxx2Kugyv-O2P_BpG8H1USZAhp
To: /kaggle/working/um_uh.mp3
100%|████████████████████████████████████████| 476k/476k [00:00<00:00, 83.6MB/s]


In [4]:
# !gdown --id 1WfnmYuP58qC_68dKIcYgqqbPc_Usm6yU

In [ ]:
# https://drive.google.com/file/d/1WfnmYuP58qC_68dKIcYgqqbPc_Usm6yU/view?usp=share_link

In [ ]:
# https://drive.google.com/drive/folders/1fLm772uCjwhDD6GJxigvDPNGATLvtBiD?usp=share_link

In [5]:
# !ls

In [6]:
import json
import subprocess
import soundfile
import torchaudio
import librosa
import torch.nn as nn
import numpy as np

from transformers import AutoConfig, Wav2Vec2Processor

In [11]:
SAMPLE_RATE = 16000

In [12]:
def ffmpeg_convert(input_audiofile, output_audiofile, sr=SAMPLE_RATE):
    """
    Convert an audio file to a resampled audio file with the desired
    sampling rate specified by `sr`.
    Parameters
    ----------
    input_audiofile : string
            Path to the video or audio file to be resampled.
    output_audiofile
            Path for saving the resampled audio file. Should have .wav extension.
    sr : int
            The sampling rate to use for resampling (e.g. 16000, 44100, 48000).
    Returns
    -------
    completed_process : subprocess.CompletedProcess
            A process completion object. If completed_process.returncode is 0 it
            means the process completed successfully. 1 means it failed.
    """

    # fmpeg command
    cmd = ["ffmpeg", "-i", input_audiofile, "-ac", "1", "-af", "aresample=resampler=soxr", "-ar", str(sr), "-y", output_audiofile]
    # print(' '.join(cmd))
    # return
    completed_process = subprocess.run(cmd)

    # confrim process completed successfully
    assert completed_process.returncode == 0

    # confirm new file has desired sample rate
    assert soundfile.info(output_audiofile).samplerate == sr

In [13]:
# global name_id = 0
def cut_file(src_filepath, tar_filepath, start_time, end_time):
#     tar_filepath = src_filepath_wo + str(name_id)
    cut_cmd = ["ffmpeg", "-i", src_filepath, "-ss", str(start_time), "-to", str(end_time), tar_filepath]
    print(' '.join(cut_cmd))
    completed_process = subprocess.run(cut_cmd)
    assert completed_process.returncode == 0
#     return tar_filepath

In [14]:
json_file = open("response_1680775446119.json")
data_str = json.load(json_file)
data = json.loads(data_str)

In [15]:
# !rm -rf gaps
!mkdir gaps_mp3
!mkdir gaps_wav

In [ ]:
gaps = []
start_gap = 0
end_gap = 0
cnt = 1
for word in data['words']:
#     data[]
    
    end_gap = word['start']
    if end_gap - start_gap < 0.2:
        continue
    cut_file('um_uh.mp3', f'gaps_mp3/um_uh_{cnt}.mp3', start_gap, end_gap)
    ffmpeg_convert(f'gaps_mp3/um_uh_{cnt}.mp3', f'gaps_wav/um_uh_{cnt}.wav')
    print("gap: ", start_gap, end_gap)
    print("word: ", word['start'], word['end'])
    start_gap = word['end']
    cnt += 1

In [17]:
# !ffmpeg -i um_uh.mp3 -ss 0 -to 1.3 gaps/um_uh_1.mp3

In [19]:
# !tree

In [21]:
# !zip -r gaps gaps_wav 

In [41]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [42]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
#         print("hidden_states.shape: ", hidden_states.shape)
        logits = self.classifier(hidden_states)
#         print("logits: ", logits)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [48]:
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
pooling_mode = "mean"
label_list = ['Breath', 'Laughter', 'Music', 'Uh', 'Um', 'Words']
num_labels = 6

In [49]:
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [59]:
label2id = {label: i for i, label in enumerate(label_list)},
label2id

({'Breath': 0, 'Laughter': 1, 'Music': 2, 'Uh': 3, 'Um': 4, 'Words': 5},)

In [50]:
model_loaded = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [51]:
# model link
!gdown --id 1cPMPJPgHZ_s5jT-GU65mwCkjBSckNB0j

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=1cPMPJPgHZ_s5jT-GU65mwCkjBSckNB0j
From (redirected): https://drive.google.com/uc?id=1cPMPJPgHZ_s5jT-GU65mwCkjBSckNB0j&confirm=t&uuid=9e0f8862-ba3e-4228-9510-1a74f3ebfd4a
To: /kaggle/working/interjections_clf_cpu.pt
100%|███████████████████████████████████████| 1.26G/1.26G [00:06<00:00, 188MB/s]


In [52]:
model_loaded.load_state_dict(torch.load('interjections_clf_cpu.pt'))

<All keys matched successfully>

In [53]:
# model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)

In [54]:
def speech_file_to_array_fn(file_path):
    speech_array, sampling_rate = torchaudio.load(file_path)
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

#     batch["speech"] = 
    return speech_array


def predict(speech_array, model, device):
    features = processor(speech_array, sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True, return_attention_mask=True)

#     print(features)
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)
    
    with torch.no_grad():
        logits = model(input_values, attention_mask).logits #attention_mask

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
#     batch["predicted"] = pred_ids
    return pred_ids

In [56]:
device = 'cpu'

array([3])

In [63]:
for idx in range(1, 15):
    speech_arr = speech_file_to_array_fn(f"gaps_wav/um_uh_{idx}.wav")
    print(idx, ": ", predict(speech_arr, model_loaded, device))

1 :  [3]
2 :  [5]
3 :  [4]
4 :  [5]
5 :  [4]
6 :  [5]
7 :  [5]
8 :  [5]
9 :  [3]
10 :  [5]
11 :  [5]
12 :  [5]
13 :  [2]
14 :  [5]


In [66]:
label2id = {label: i for i, label in enumerate(label_list)},
label2id

({'Breath': 0, 'Laughter': 1, 'Music': 2, 'Uh': 3, 'Um': 4, 'Words': 5},)